In [1]:
import pandas as pd
import numpy as np

In [3]:
data = 'https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv'
!wget $data

--2025-10-14 15:36:32--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80876 (79K) [text/plain]
Saving to: ‘course_lead_scoring.csv’

course_lead_scoring 100%[===================>]  78.98K  --.-KB/s    in 0.002s  

2025-10-14 15:36:32 (42.2 MB/s) - ‘course_lead_scoring.csv’ saved [80876/80876]



In [7]:
df = pd.read_csv('course_lead_scoring.csv')
df.columns = df.columns.str.lower().str.replace(' ','_')
df.head()

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [15]:
#check if there are any categorical values that Nan and convert them to 'NA'
cat_cols = df.select_dtypes(include=['object', 'category']).columns
#replace categorical features of nan to NA
cat_features = df[cat_cols].fillna('NA')
cat_features.head()

,lead_source,industry,employment_status,location
0,paid_ads,NA,unemployed,south_america
1,social_media,retail,employed,south_america
2,events,healthcare,unemployed,australia
3,paid_ads,retail,NA,australia
4,referral,education,self_employed,europe


In [19]:
#check if there are any numberical features that are Nan and convert them to 0
num_cols = df.select_dtypes(include=['number']).columns
#replace numerical features of nan to 0
num_features = df[num_cols].fillna(0.0)
num_features.head()

,number_of_courses_viewed,annual_income,interaction_count,lead_score,converted
0,1,79450.0,4,0.94,1
1,1,46992.0,1,0.80,0
2,5,78796.0,3,0.69,1
3,2,83843.0,1,0.87,0
4,3,85012.0,3,0.62,1


In [22]:
cat_features['industry'].mode()

0    retail
Name: industry, dtype: object